## Walkthrough of the test env code

In [46]:
import time
import sys

import numpy as np
import pandas as pd

from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS

Import files from sibling a directory

In [47]:
sys.path.append("..")

import model.clean as cl
import model.model_trainer as mt
import model.model_predict as mp
from model.influx_interact import influx_class

### Step 1
Set up a local instance of influx by following the README.md

### Step 2  
`populate_influx.py`  
Populating influx with data from csvs stored in the repo

In [48]:
PATH_TO_CSVS = "../../data/labelled-skyspark-data/"
CSVS_TO_LOAD = [
    "CEC_compiled_data_1b_updated.csv",
    "CEC_compiled_data_2b_updated.csv",
    "CEC_compiled_data_3b_updated.csv",
    "CEC_compiled_data_4b_updated.csv",
    "CEC_compiled_data_5b_updated.csv",
]

Set up influx connection

In [49]:
# as set up in the docker-compose
token = "mytoken"
org = "UBC"
bucket = "MDS2021"

# set up influx
client = InfluxDBClient(url="http://localhost:8086", token=token, timeout=999_000)
write_api = client.write_api(write_options=SYNCHRONOUS)

Loop over each csv, read it then write it into influx  

Important note: If the influx write times out, re-run and it should work on second try.

In [51]:
for csv in CSVS_TO_LOAD:

    # load and set up dataframes
    df = pd.read_csv(PATH_TO_CSVS + csv, parse_dates=["Datetime"])
    df.rename(columns={"Value": "val_num"}, inplace=True)
    df.rename(columns={"ID": "uniqueID"}, inplace=True)
    df.rename(columns={"Anomaly": "AH"}, inplace=True)
    df["navName"] = "Energy"
    df["siteRef"] = "Campus Energy Centre"
    df.set_index("Datetime", drop=True, inplace=True)
    df = df.drop(["AH"], axis=1)

    print("writing: {}".format(csv))
    # write values
    write_api.write(
        bucket,
        org,
        record=df,
        data_frame_measurement_name="READINGS",
        data_frame_tag_columns=["uniqueID", "navName", "siteRef"],
    )
    time.sleep(5)

writing: CEC_compiled_data_1b_updated.csv
writing: CEC_compiled_data_2b_updated.csv
writing: CEC_compiled_data_3b_updated.csv
writing: CEC_compiled_data_4b_updated.csv
writing: CEC_compiled_data_5b_updated.csv


Look at the `df` object to see what was written to influx

In [52]:
df.head()

,val_num,uniqueID,navName,siteRef
Datetime,,,,
2020-01-01 07:45:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:00:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:15:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:30:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre
2020-01-01 08:45:00,2.9,Campus Energy Centre Boiler B-1 Exhaust O2,Energy,Campus Energy Centre


Now sensor data has been written to influx's READINGS bucket  
This simulates what UDL's influx will look like

### Step 3
`test_env_scheduled_training.py`  
Run the training file

Subset the training data for fast dev iterations

In [53]:
TESTING = True

Setup thresholds for anomaly detection.  
If a training example's loss is greater than the set threshold it will be flagged as anomalous

In [54]:
THRESHOLDS = {
    "Campus Energy Centre Campus HW Main Meter Power": 0.09,
    "Campus Energy Centre Boiler B-1 Exhaust O2": 0.019,
    "Campus Energy Centre Boiler B-1 Gas Pressure": 0.0725,
    "Campus Energy Centre Campus HW Main Meter Entering Water Temperature": 0.02938,
    "Campus Energy Centre Campus HW Main Meter Flow": 0.043,
}

Pick a end time for the readings to be used for training  
In the real implementation there will be no end_time for the training data, i.e it will train on all the data

In [55]:
END_TIME = 1613109600

Explore training with removing anomalous records, i.e only train on normal data for this specific sensor data

In [56]:
REMOVE_ANOMALOUS = True
REMOVE_ANOMALOUS_DATA = [
    "Campus Energy Centre Campus HW Main Meter Entering Water Temperature"
]

Influx connector setup

In [57]:
model_path = "./test_env_models/"
scaler_path = "./test_env_standardizers/"

# set up for influx
token = "mytoken"
org = "UBC"
bucket = "MDS2021"
url = "http://localhost:8086"

influx_conn = influx_class(
    org=org,
    url=url,
    bucket=bucket,
    token=token,
)

Read the training data from influx

In [58]:
influx_read_df = influx_conn.make_query(
    location="Campus Energy Centre",
    measurement="READINGS",
    end=END_TIME,
)

Split the data based on sensor ID

In [59]:
main_bucket = cl.split_sensors(influx_read_df)

the `main_bucket` object is a dict with the name of the sensor as the key and then the value is another dict of data objects

In [60]:
main_bucket.keys()

dict_keys(['Campus Energy Centre Boiler B-1 Exhaust O2', 'Campus Energy Centre Boiler B-1 Gas Pressure', 'Campus Energy Centre Campus HW Main Meter Entering Water Temperature', 'Campus Energy Centre Campus HW Main Meter Flow', 'Campus Energy Centre Campus HW Main Meter Power'])

### The following cell for training:
1) Iterates over each the data sets (line 1)  
2) Removes anomalous data if the data set has been specified (lines 4-23)  
3) Standardizes the values for training and saves the standardizer for the prediction step (lines 25-30)  
4) Subsets the data for faster training if specified (lines 32-33)  
5) Sequences the values into windows for the LSTM (lines 35-37)  
6) Fits and saves the model (line 42)  
7) Writes the resulting data (that contains: value, anomalies detected manually, and anomalies detected in training) to influx (lines 44-57)

In [61]:
for key, df in main_bucket.items():
    print("Training for : {}".format(key))

    # removes anomalies to only train on normal data
    if REMOVE_ANOMALOUS:
        if key in REMOVE_ANOMALOUS_DATA:
            PATH_TO_CSVS = "../../data/labelled-skyspark-data/"
            csv = "CEC_compiled_data_2b_updated.csv"
            df_with_manual_anomaly = pd.read_csv(
                PATH_TO_CSVS + csv, parse_dates=["Datetime"]
            )
            df_with_manual_anomaly["Datetime"] = pd.to_datetime(
                df_with_manual_anomaly["Datetime"], utc=True
            )
            df = df.merge(
                df_with_manual_anomaly[["Datetime", "Anomaly"]],
                how="left",
                left_on="DateTime",
                right_on="Datetime",
            )
            df = df.loc[df["Anomaly"] == False]
            df = df.drop(columns=["DateTime"], axis=1)
            am_df.rename(columns={"Anomaly": "manual_anomaly"}, inplace=True)

    # creates standardized column for each sensor in main bucket
    df["Stand_Val"] = cl.std_val_train(
        df[["Value"]],
        main_bucket[key]["ID"].any(),
        scaler_path,
    )

    if TESTING:
        df = df.tail(5000)

    # creates arrays for sliding windows
    x_train, y_train = mt.create_sequences(df["Stand_Val"], df["Stand_Val"])
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    normal_dict = cl.model_parser(df, x_train, y_train)

    threshold = THRESHOLDS[key]
    mt.fit_models(normal_dict, model_path, threshold)

    # for writing AM to influx
    am_df = normal_dict[key]["train_score_df"]
    am_df.rename(columns={"anomaly": "model_anomaly"}, inplace=True)
    am_df.rename(columns={"ID": "uniqueID"}, inplace=True)
    am_df.rename(columns={"Datetime": "DateTime"}, inplace=True)
    am_df["val_num"] = df["Value"].iloc[x_train.shape[1] :]
    # only if it hasnt already been created earlier
    if "manual_anomaly" not in set(am_df.columns):
        am_df["manual_anomaly"] = False
    am_df.set_index("DateTime", drop=True, inplace=True)
    am_df = am_df[["uniqueID", "model_anomaly", "val_num", "manual_anomaly"]]

    influx_conn.write_data(am_df, "TRAINING_ANOMALY")

Training for : Campus Energy Centre Boiler B-1 Exhaust O2
Epoch 1/100
141/141 [==============================] - 6s 23ms/step - loss: 0.0010 - val_loss: 1.4035
Epoch 2/100
141/141 [==============================] - 2s 18ms/step - loss: 3.5567e-04 - val_loss: 1.4194
Epoch 3/100
141/141 [==============================] - 3s 20ms/step - loss: 4.2476e-04 - val_loss: 1.4245
Epoch 4/100
141/141 [==============================] - 3s 21ms/step - loss: 1.1841e-04 - val_loss: 1.4251
Training for : Campus Energy Centre Boiler B-1 Gas Pressure
Epoch 1/100
141/141 [==============================] - 6s 24ms/step - loss: 0.2650 - val_loss: 0.3818
Epoch 2/100
141/141 [==============================] - 2s 18ms/step - loss: 0.2121 - val_loss: 0.3204
Epoch 3/100
141/141 [==============================] - 3s 20ms/step - loss: 0.2086 - val_loss: 0.3192
Epoch 4/100
141/141 [==============================] - 3s 21ms/step - loss: 0.2052 - val_loss: 0.3081
Epoch 5/100
141/141 [==============================] -

/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Epoch 1/100
141/141 [==============================] - 6s 22ms/step - loss: 0.2807 - val_loss: 0.3015
Epoch 2/100
141/141 [==============================] - 3s 20ms/step - loss: 0.1897 - val_loss: 0.2515
Epoch 3/100
141/141 [==============================] - 3s 22ms/step - loss: 0.1589 - val_loss: 0.1586
Epoch 4/100
141/141 [==============================] - 3s 21ms/step - loss: 0.1516 - val_loss: 0.1408
Epoch 5/100
141/141 [==============================] - 3s 21ms/step - loss: 0.1418 - val_loss: 0.1738
Epoch 6/100
141/141 [==============================] - 3s 21ms/step - loss: 0.1393 - val_loss: 0.1361
Epoch 7/100
141/141 [==============================] - 3s 22ms/step - loss: 0.1366 - val_loss: 0.1177
Epoch 8/100
141/141 [==============================] - 3s 21ms/step - loss: 0.1308 - val_loss: 0.1178
Epoch 9/100
141/141 [==============================] - 3s 22ms/step - loss: 0.1306 - val_loss: 0.1237
Epoch 10/100
141/141 [==============================] - 3s 23ms/step - loss: 0.126

### Step 4
`test_env_scheduled_predictor.py`  
Create predictions and write to influx

Set up start and end times for the prediction data set in the test env.  
In the real implementation END_TIME would be `now()` and START_TIME would be `now() - 1d`

In [62]:
# END TIME FOR TRAINING SET BECOMES PREDICTING'S START TIME
START_TIME = 1613109600
END_TIME = 1613196000

Read the data to be predicted on from influx

In [63]:
print("reading data from influx")
influx_read_df = influx_conn.make_query(
    location="Campus Energy Centre",
    measurement="READINGS",
    start=START_TIME,
    end=END_TIME,
)

reading data from influx


Split based on data name

In [64]:
main_bucket = cl.split_sensors(influx_read_df)

### The following cell for predicting:
1) Iterates over each the data sets (line 1)  
2) Standardizes the values for training by loading the standardizer from the training step (lines 2-6)     
3) Sequences the values into windows for the LSTM and other reshaping for the prediction step(lines 8-14)  
4) Creates predictions for the data and returns the prediction object (lines 16-24)   
5) Shapes the prediction object for writing back to influx (a dataframe that contains, value, and the prediction of anomalous) (lines 26-34) 

In [65]:
for key, df in main_bucket.items():
    main_bucket[key]["Stand_Val"] = cl.std_val_predict(
        main_bucket[key][["Value"]],
        main_bucket[key]["ID"].any(),
        scaler_path,
    )

    # creates arrays for sliding windows
    x_train, y_train = mt.create_sequences(
        main_bucket[key]["Stand_Val"], main_bucket[key]["Stand_Val"]
    )
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    timestamps = df["DateTime"].tail(len(df) - x_train.shape[1]).values
    threshold = THRESHOLDS[key]

    # predicting and prediction formatting
    pred = mp.make_prediction(
        key,
        x_train,
        timestamps,
        threshold,
        model_path,
    )
    ar_df = pd.DataFrame.from_dict(pred["data"])

    # prep for writing
    ar_df.rename(columns={"anomaly": "realtime_anomaly"}, inplace=True)
    ar_df.rename(columns={"Timestamp": "DateTime"}, inplace=True)
    ar_df["uniqueID"] = key
    ar_df.set_index("DateTime", drop=True, inplace=True)
    ar_df["val_num"] = df["Value"].tail(len(df) - x_train.shape[1]).values
    ar_df = ar_df[["uniqueID", "val_num", "realtime_anomaly"]]

    influx_conn.write_data(ar_df, "PREDICT_ANOMALY")

The test environment will now have three readings:  
READINGS: the raw data  
TRAINING_ANOMALY: data with anomalies flag manually and during the training  
PREDICT_ANOMALY: data with anomalies detected by the prediction step  

As well as a standardizer and a model for each data set.